In [1]:
import pymysql.cursors
import pandas as pd

conn = pymysql.connect(host='localhost', 
                       user='root', 
                       password='root', 
                       db='zio', 
                       charset='utf8mb4', 
                       autocommit=True, 
                       cursorclass=pymysql.cursors.DictCursor)

def get_investors_volume() :
    cur = conn.cursor()
    sql = "select DATEON, PERSONAL, FOREIGNER, COMPANY, FINANCE, INSURANCE, TOOSIN, BANK, ETC_FIN, GOV_FUND, ETC_FUND from CRAWL_INVESTOR_DAILY_VOLUME where TYPE_CODE='02'"
    cur.execute(sql)
    result = cur.fetchall()
    cur.close()
    df = pd.DataFrame(result)
    df = df.set_index('DATEON')
    return df

def exec_query(sql) -> pd.DataFrame :
    cur = conn.cursor()
    cur.execute(sql)
    result = cur.fetchall()
    # print(result.count())
    cur.close()
    df = pd.DataFrame(result)
    return df

# def end_price(sql = "select * from CRWAL_INVESTOR_STOCK_TYPE") :
#     return exec_query(sql)

def get_daily_kosdaq() :
    sql = 'select * from CRAWL_MARKET_DAILY_DATA'
    return exec_query(sql)

def get_daily_investors_volume() :
    sql = "select DATEON, PERSONAL, FOREIGNER, COMPANY, FINANCE, INSURANCE, TOOSIN, BANK, ETC_FIN, GOV_FUND, ETC_FUND from CRAWL_INVESTOR_DAILY_VOLUME where TYPE_CODE='02'"
    return exec_query(sql)

# get_investors_volume()

#end_price()
get_daily_kosdaq()

# from pycaret.datasets import get_data
# get_data('insurance')

,TYPE_CODE,DATEON,CLOSING_PRICE,UP_DOWN_PER,VOLUME,VOLUME_AMT
0,KOSDAQ,1996.07.04,1018.40,1.30,235.0,5247.0
1,KOSDAQ,1996.07.05,1037.80,1.90,59.0,894.0
2,KOSDAQ,1996.07.06,1048.80,1.06,34.0,338.0
3,KOSDAQ,1996.07.08,1048.20,-0.06,39.0,822.0
4,KOSDAQ,1996.07.09,1060.10,1.14,45.0,589.0
...,...,...,...,...,...,...
7047,KOSDAQ,2024.08.19,777.47,-1.13,1135630.0,7653030.0
7048,KOSDAQ,2024.08.20,787.44,1.28,997056.0,7990590.0
7049,KOSDAQ,2024.08.21,779.87,-0.96,1073930.0,8370580.0
7050,KOSDAQ,2024.08.22,773.47,-0.82,950992.0,9216820.0


In [2]:
investors_volume = get_daily_investors_volume()
# investors_volume = investors_volume.sort_value(by=['DATEON'], axis=0, ascending=True)
# investors_volume.reset_index()
investors_volume = investors_volume.set_index('DATEON')
investors_volume.sort_index(ascending=False, inplace=True)

daily_kosdaq = get_daily_kosdaq()
daily_kosdaq['DATEON'] = daily_kosdaq['DATEON'].str[2:]
daily_kosdaq.set_index('DATEON').drop(labels = ['TYPE_CODE'], axis=1)

import pandas as pd

merged = pd.merge(left=investors_volume, right=daily_kosdaq, how='inner', on='DATEON')

merged.set_index('DATEON')
merged.sort_index(ascending=True, inplace=True)
merged = merged.set_index('DATEON')
merged = merged.drop(labels = ['TYPE_CODE'], axis=1)
merged

,PERSONAL,FOREIGNER,COMPANY,FINANCE,INSURANCE,TOOSIN,BANK,ETC_FIN,GOV_FUND,ETC_FUND,CLOSING_PRICE,UP_DOWN_PER,VOLUME,VOLUME_AMT
DATEON,,,,,,,,,,,,,,
24.08.23,-536.0,982.0,-107.0,56.0,37.0,-158.0,-8.0,-179.0,145.0,-339.0,773.62,0.02,876114.0,7724720.0
24.08.22,1745.0,-817.0,-746.0,35.0,6.0,-635.0,-7.0,-149.0,4.0,-183.0,773.47,-0.82,950992.0,9216820.0
24.08.21,3639.0,-3343.0,-47.0,276.0,35.0,-414.0,-4.0,4.0,57.0,-249.0,779.87,-0.96,1073930.0,8370580.0
24.08.20,-1106.0,1005.0,156.0,218.0,25.0,-204.0,-5.0,-54.0,176.0,-55.0,787.44,1.28,997056.0,7990590.0
24.08.19,1394.0,-1130.0,-134.0,-59.0,31.0,-43.0,0.0,-54.0,-8.0,-130.0,777.47,-1.13,1135630.0,7653030.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05.01.25,275.0,-76.0,-199.0,-24.0,-17.0,4.0,-34.0,-11.0,-117.0,0.0,467.29,-1.15,691510.0,1931120.0
05.01.24,321.0,-124.0,-196.0,-33.0,12.0,-48.0,22.0,-15.0,-135.0,0.0,472.75,2.63,603670.0,1756330.0
05.01.21,235.0,-177.0,-58.0,3.0,-17.0,-49.0,24.0,-2.0,-17.0,0.0,460.62,2.22,607820.0,1587980.0


In [3]:
def create_ex_columns(df , name: str) :
    # print(name + ' newly added ..')
    df[name + '_SUM_D5'] = df[name].rolling(5).sum().shift(1)
    df[name + '_AVG_D5'] = df[name].rolling(5).mean().shift(1)
    df[name + '_STD_D5'] = df[name].rolling(5).std().shift(1)
    df[name + '_SUM_D10'] = df[name].rolling(10).sum().shift(1)
    df[name + '_AVG_D10'] = df[name].rolling(10).mean().shift(1)
    df[name + '_STD_D10'] = df[name].rolling(10).std().shift(1)
    df[name + '_SUM_D20'] = df[name].rolling(20).sum().shift(1)
    df[name + '_AVG_D20'] = df[name].rolling(20).mean().shift(1)
    df[name + '_STD_D20'] = df[name].rolling(20).std().shift(1)

feature_cols = ['PERSONAL', 'FOREIGNER', 'COMPANY', 'FINANCE', 'INSURANCE', 'TOOSIN', 'BANK', 'ETC_FIN', 'GOV_FUND', 'ETC_FUND',
                'CLOSING_PRICE', 'UP_DOWN_PER', 'VOLUME', 'VOLUME_AMT']

for col in feature_cols:
    create_ex_columns(merged, col)
    
# merged['PRED_OUT'] = merged['CLOSING_PRICE'].shift(1)
merged['PRED_OUT_D3'] = merged['CLOSING_PRICE'].rolling(3).mean().shift(1)
# merged[['CLOSING_PRICE', 'PRED_OUT', 'PRED_OUT_D3']]
merged = merged.dropna()
merged.sort_index(ascending=False)
merged                          

/var/folders/vs/tzb5k9f92k14smyyxl44gdwh0000gn/T/ipykernel_64362/4029696418.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name + '_STD_D20'] = df[name].rolling(20).std().shift(1)
/var/folders/vs/tzb5k9f92k14smyyxl44gdwh0000gn/T/ipykernel_64362/4029696418.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name + '_SUM_D5'] = df[name].rolling(5).sum().shift(1)
/var/folders/vs/tzb5k9f92k14smyyxl44gdwh0000gn/T/ipykernel_64362/4029696418.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

,PERSONAL,FOREIGNER,COMPANY,FINANCE,INSURANCE,TOOSIN,BANK,ETC_FIN,GOV_FUND,ETC_FUND,...,VOLUME_AMT_SUM_D5,VOLUME_AMT_AVG_D5,VOLUME_AMT_STD_D5,VOLUME_AMT_SUM_D10,VOLUME_AMT_AVG_D10,VOLUME_AMT_STD_D10,VOLUME_AMT_SUM_D20,VOLUME_AMT_AVG_D20,VOLUME_AMT_STD_D20,PRED_OUT_D3
DATEON,,,,,,,,,,,,,,,,,,,,,
24.07.25,959.0,-1395.0,506.0,2059.0,-83.0,-1048.0,-15.0,-42.0,-366.0,-69.0,...,30066920.0,6013384.0,530804.850044,68566060.0,6856606.0,1.169718e+06,147118810.0,7355940.5,1.046436e+06,803.110000
24.07.24,460.0,-320.0,208.0,82.0,23.0,-5.0,3.0,-21.0,127.0,-348.0,...,31265570.0,6253114.0,780692.180875,68675470.0,6867547.0,1.173480e+06,146673490.0,7333674.5,1.042908e+06,800.946667
24.07.23,-1807.0,1222.0,558.0,128.0,46.0,499.0,-5.0,-6.0,-105.0,27.0,...,31137040.0,6227408.0,763346.728636,67933780.0,6793378.0,1.164094e+06,144049410.0,7202470.5,9.548749e+05,803.033333
24.07.22,2592.0,-1809.0,-869.0,123.0,-33.0,-580.0,-20.0,-5.0,-354.0,86.0,...,32211980.0,6442396.0,912459.906122,66889050.0,6688905.0,1.046631e+06,143016130.0,7150806.5,9.154881e+05,807.886667
24.07.19,-225.0,-312.0,650.0,-185.0,38.0,706.0,-1.0,-4.0,96.0,-113.0,...,33514900.0,6702980.0,687739.033210,64494990.0,6449499.0,6.354720e+05,141697700.0,7084885.0,8.991576e+05,812.110000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05.01.25,275.0,-76.0,-199.0,-24.0,-17.0,4.0,-34.0,-11.0,-117.0,0.0,...,9469240.0,1893848.0,257540.243205,16780760.0,1678076.0,2.963113e+05,35715190.0,1785759.5,2.753623e+05,467.823333
05.01.24,321.0,-124.0,-196.0,-33.0,12.0,-48.0,22.0,-15.0,-135.0,0.0,...,9747110.0,1949422.0,219867.780996,17121730.0,1712173.0,3.045722e+05,35893720.0,1794686.0,2.771186e+05,467.410000
05.01.21,235.0,-177.0,-58.0,3.0,-17.0,-49.0,24.0,-2.0,-17.0,0.0,...,9783470.0,1956694.0,210795.668077,17560600.0,1756060.0,2.711639e+05,35836330.0,1791816.5,2.772082e+05,469.510000


In [4]:
train = merged.sample(frac=0.95, random_state=786)
test = merged.drop(train.index)
# train.reset_index(inplace=True, drop=False)
# test.reset_index(inplace=True, drop=False)

# train.sort_index(ascending=False)
test.sort_index(ascending=False)

,PERSONAL,FOREIGNER,COMPANY,FINANCE,INSURANCE,TOOSIN,BANK,ETC_FIN,GOV_FUND,ETC_FUND,...,VOLUME_AMT_SUM_D5,VOLUME_AMT_AVG_D5,VOLUME_AMT_STD_D5,VOLUME_AMT_SUM_D10,VOLUME_AMT_AVG_D10,VOLUME_AMT_STD_D10,VOLUME_AMT_SUM_D20,VOLUME_AMT_AVG_D20,VOLUME_AMT_STD_D20,PRED_OUT_D3
DATEON,,,,,,,,,,,,,,,,,,,,,
24.07.18,54.0,823.0,-744.0,1004.0,-196.0,-1274.0,-8.0,-23.0,-246.0,-133.0,...,34711100.0,6942220.0,345526.059509,64778020.0,6477802.0,646478.406768,140874170.0,7043708.5,8.905861e+05,816.933333
24.07.10,1704.0,-514.0,-992.0,-647.0,3.0,-96.0,1.0,-38.0,-217.0,-198.0,...,36871090.0,7374218.0,606938.269695,71265780.0,7126578.0,519724.602603,139941250.0,6997062.5,8.932498e+05,851.890000
24.07.08,-367.0,792.0,-490.0,-591.0,1.0,56.0,-10.0,-16.0,70.0,65.0,...,38940990.0,7788198.0,899053.760695,74385170.0,7438517.0,748437.918914,141274220.0,7063711.0,9.654545e+05,857.130000
24.07.01,-1109.0,104.0,1167.0,960.0,43.0,241.0,-12.0,-37.0,-28.0,-162.0,...,44556360.0,8911272.0,704341.580961,83497350.0,8349735.0,964412.137168,151153510.0,7557675.5,1.167059e+06,835.606667
24.05.02,968.0,-689.0,-152.0,159.0,-76.0,-119.0,-3.0,-50.0,-62.0,-127.0,...,44602280.0,8920456.0,782219.419300,90257620.0,9025762.0,687841.778622,185445310.0,9272265.5,7.961814e+05,869.756667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05.12.26,-35.0,26.0,9.0,30.0,19.0,-76.0,-8.0,-17.0,60.0,0.0,...,12611450.0,2522290.0,515067.994686,29085660.0,2908566.0,567778.601064,56031570.0,2801578.5,4.530046e+05,693.760000
05.10.04,-615.0,449.0,165.0,61.0,44.0,61.0,30.0,-12.0,-19.0,0.0,...,11205500.0,2241100.0,115846.456571,22959930.0,2295993.0,151976.140894,43069420.0,2153471.0,2.019068e+05,590.380000
05.06.14,106.0,37.0,-143.0,-34.0,18.0,-12.0,-34.0,12.0,-93.0,0.0,...,8856400.0,1771280.0,112836.743129,17173580.0,1717358.0,169712.937751,34780600.0,1739030.0,1.490829e+05,489.236667


In [5]:
from pycaret.regression import *

s = setup(data = train, target='PRED_OUT_D3', session_id=123, normalize=True, normalize_method='minmax')

ImportError: cannot import name 'interp' from 'scipy' (/Users/younggonkim/.conda/envs/myFirstCrawl/lib/python3.9/site-packages/scipy/__init__.py)

In [7]:
best = compare_models(include=['et', 'rf', 'lightgbm'])


NameError: name 'compare_models' is not defined

In [11]:
evaluate_model(best)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [12]:
pred_holdout = predict_model(best)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,2.6142,16.1706,4.0213,0.9994,0.0073,0.0042


In [14]:
interpret_model(best)

ModuleNotFoundError: 
'shap' is a soft dependency and not included in the pycaret installation. Please run: `pip install shap` to install.
Alternately, you can install this by running `pip install pycaret[analysis]`